In [3]:
import rasterio
import numpy as np
import cv2
from skimage.measure import block_reduce

def read_band(path, band_index=1):
    with rasterio.open(path) as src:
        band = src.read(band_index)
        profile = src.profile
    return band, profile

def read_first_band(path):
    with rasterio.open(path) as src:
        data = src.read(1)
        profile = src.profile
    return data, profile

def write_tiff(path, data, profile):
    profile.update(dtype='float32', count=1)
    with rasterio.open(path, 'w', **profile) as dst:
        dst.write(data.astype(np.float32), 1)

def mean_pooling(img, factor):
    return block_reduce(img, block_size=(factor, factor), func=np.mean)

def upscale_to_shape(img, target_shape):
    return cv2.resize(img, (target_shape[1], target_shape[0]), interpolation=cv2.INTER_LINEAR)

# === File paths ===
tiff_25km_path = '2003_02_TWS25.tif'
tiff_5km_path = '2003-02-01_TWS5.tiff'

# === Step 1: Extract Band 7 from 25km ===
band7_25km, profile_25km = read_band(tiff_25km_path, band_index=7)

# === Step 2: Read 5km TIFF (assumes single-band or first band used) ===
img_5km, profile_5km = read_first_band(tiff_5km_path)

# === Step 3: Confirm scaling factor ===
factor_y = img_5km.shape[0] // band7_25km.shape[0]
factor_x = img_5km.shape[1] // band7_25km.shape[1]
assert factor_x == factor_y, "Non-square or mismatched scaling!"
factor = factor_x

# === Step 4: Downsample 5km to 25km using mean pooling ===
img_5km_to_25km = mean_pooling(img_5km, factor)

# === Step 5: Compute residuals ===
residuals_25km = band7_25km - img_5km_to_25km

# === Step 6: Upsample residuals to 5km resolution ===
residuals_5km = upscale_to_shape(residuals_25km, img_5km.shape)

# === Step 7: Add residuals to original 5km image ===
corrected_5km = img_5km + residuals_5km

# === Step 8: Save result ===
write_tiff('corrected_5km.tif', corrected_5km, profile_5km)


ValueError: operands could not be broadcast together with shapes (45,54) (56,66) 

In [4]:
import rasterio
import numpy as np
import cv2

def read_tiff_band(path, band_index=1):
    with rasterio.open(path) as src:
        data = src.read(band_index)  # Band index starts at 1
        profile = src.profile
    return data, profile

def write_tiff(path, data, profile):
    with rasterio.open(path, 'w', **profile) as dst:
        dst.write(data, 1)

# === Step 1: Read band 7 from 25km TIFF and band 1 from 5km TIFF ===
tiff_25km_path = '2003_02_TWS25.tif'
tiff_5km_path = '2003-02-01_TWS5.tiff'

data_25km, profile_25km = read_tiff_band(tiff_25km_path, band_index=7)
data_5km, profile_5km = read_tiff_band(tiff_5km_path, band_index=1)

# === Step 2: Downscale 5km image to match 25km resolution ===
target_shape_25km = data_25km.shape
downscaled_5km_to_25km = cv2.resize(data_5km, (target_shape_25km[1], target_shape_25km[0]), interpolation=cv2.INTER_AREA)

# === Step 3: Compute residuals ===
residual_25km = data_25km - downscaled_5km_to_25km

# === Step 4: Upscale residuals back to 5km resolution ===
target_shape_5km = data_5km.shape
residual_upsampled_5km = cv2.resize(residual_25km, (target_shape_5km[1], target_shape_5km[0]), interpolation=cv2.INTER_LINEAR)

# === Step 5: Add residuals to original 5km data ===
corrected_5km = data_5km + residual_upsampled_5km

# === Optional: Save corrected 5km data ===
output_path = 'corrected_5km_with_residuals.tif'
profile_5km.update(dtype='float32')

write_tiff(output_path, corrected_5km.astype(np.float32), profile_5km)


In [1]:
import os
import rasterio
import numpy as np
import cv2

# Utility functions
def read_tiff_band(path, band_index=1):
    with rasterio.open(path) as src:
        data = src.read(band_index)
        profile = src.profile
    return data, profile

def write_tiff(path, data, profile):
    with rasterio.open(path, 'w', **profile) as dst:
        dst.write(data, 1)

# Define directories
dir_25km = '25kmTWScombined_monthly_tiffs'
dir_5km = 'TWS_Predictions'
output_dir = 'Corrected_5km_Residuals'
os.makedirs(output_dir, exist_ok=True)

# Process all 25km files
for file_25km in sorted(os.listdir(dir_25km)):
    if not file_25km.lower().endswith('.tif'):
        continue

    base_name = os.path.splitext(file_25km)[0]  # Example: '2003_02'
    corresponding_5km_name = base_name.replace('_', '-') + '-01.tiff'  # Example: '2003-02-01.tiff'

    path_25km = os.path.join(dir_25km, file_25km)
    path_5km = os.path.join(dir_5km, corresponding_5km_name)

    if not os.path.exists(path_5km):
        print(f"Missing 5km file for {file_25km} -> Skipping.")
        continue

    print(f"Processing: 25km={file_25km}, 5km={corresponding_5km_name}")

    # Step 1: Read required bands
    data_25km, profile_25km = read_tiff_band(path_25km, band_index=7)
    data_5km, profile_5km = read_tiff_band(path_5km, band_index=1)

    # Step 2: Downscale 5km to match 25km resolution
    target_shape_25km = data_25km.shape
    downscaled_5km_to_25km = cv2.resize(data_5km, (target_shape_25km[1], target_shape_25km[0]), interpolation=cv2.INTER_AREA)

    # Step 3: Compute residuals
    residual_25km = data_25km - downscaled_5km_to_25km

    # Step 4: Upscale residuals back to 5km resolution
    target_shape_5km = data_5km.shape
    residual_upsampled_5km = cv2.resize(residual_25km, (target_shape_5km[1], target_shape_5km[0]), interpolation=cv2.INTER_LINEAR)

    # Step 5: Add residuals to original 5km data
    corrected_5km = data_5km + residual_upsampled_5km

    # Step 6: Save result
    profile_5km.update(dtype='float32')
    output_filename = f"corrected_{corresponding_5km_name.replace('.tiff', '.tif')}"
    output_path = os.path.join(output_dir, output_filename)

    write_tiff(output_path, corrected_5km.astype(np.float32), profile_5km)

print("Processing complete.")


Processing: 25km=2003_02.tif, 5km=2003-02-01.tiff
Processing: 25km=2003_03.tif, 5km=2003-03-01.tiff
Processing: 25km=2003_04.tif, 5km=2003-04-01.tiff
Processing: 25km=2003_05.tif, 5km=2003-05-01.tiff
Processing: 25km=2003_06.tif, 5km=2003-06-01.tiff
Processing: 25km=2003_07.tif, 5km=2003-07-01.tiff
Processing: 25km=2003_08.tif, 5km=2003-08-01.tiff
Processing: 25km=2003_09.tif, 5km=2003-09-01.tiff
Processing: 25km=2003_10.tif, 5km=2003-10-01.tiff
Processing: 25km=2003_11.tif, 5km=2003-11-01.tiff
Processing: 25km=2003_12.tif, 5km=2003-12-01.tiff
Processing: 25km=2004_01.tif, 5km=2004-01-01.tiff
Processing: 25km=2004_02.tif, 5km=2004-02-01.tiff
Processing: 25km=2004_03.tif, 5km=2004-03-01.tiff
Processing: 25km=2004_04.tif, 5km=2004-04-01.tiff
Processing: 25km=2004_05.tif, 5km=2004-05-01.tiff
Processing: 25km=2004_06.tif, 5km=2004-06-01.tiff
Processing: 25km=2004_07.tif, 5km=2004-07-01.tiff
Processing: 25km=2004_08.tif, 5km=2004-08-01.tiff
Processing: 25km=2004_09.tif, 5km=2004-09-01.tiff


In [2]:
import os
import rasterio
import pandas as pd
from rasterio.transform import xy
from tqdm import tqdm

# Input folder containing corrected .tif files
corrected_dir = 'Corrected_5km_Residuals'

# Output CSV file
output_csv = 'corrected_5km_data.csv'

# Storage for all rows
data_rows = []

# Iterate through all corrected .tif files
for fname in tqdm(sorted(os.listdir(corrected_dir))):
    if not fname.lower().endswith('.tif'):
        continue

    # Extract date from filename (assumes: corrected_YYYY-MM-DD.tif)
    date_str = fname.replace('corrected_', '').replace('.tif', '')

    file_path = os.path.join(corrected_dir, fname)

    with rasterio.open(file_path) as src:
        band = src.read(1)  # Read first band
        transform = src.transform

        # Iterate over each pixel
        for row in range(band.shape[0]):
            for col in range(band.shape[1]):
                value = band[row, col]
                lon, lat = xy(transform, row, col)
                data_rows.append({
                    'latitude': lat,
                    'longitude': lon,
                    'date': date_str,
                    'value': float(value)
                })

# Create DataFrame and write to CSV
df = pd.DataFrame(data_rows)
df.to_csv(output_csv, index=False)

print(f"Saved CSV with {len(df)} rows to '{output_csv}'")


100%|████████████████████████████████████████████████████████████████████████████████| 251/251 [07:55<00:00,  1.89s/it]


Saved CSV with 14644344 rows to 'corrected_5km_data.csv'


In [3]:
df.dropna(inplace=True)

# Save to CSV
df.to_csv('new_corrected_5km_data.csv', index=False)


In [15]:
df = pd.read_csv('new_corrected_5km_data.csv')
df


,latitude,longitude,date,value
0,31.553324,73.819058,2003-02-01,837.575928
1,31.553324,73.863974,2003-02-01,833.531860
2,31.553324,73.908890,2003-02-01,814.328064
3,31.553324,73.953806,2003-02-01,807.593262
4,31.508409,73.819058,2003-02-01,813.395996
...,...,...,...,...
8473447,21.671856,74.672458,2023-12-01,1166.895142
8473448,21.671856,74.717374,2023-12-01,1179.004883
8473449,21.671856,74.762290,2023-12-01,1191.114624
8473450,21.671856,74.807205,2023-12-01,1203.224365


In [16]:
import pandas as pd

df1 = pd.read_csv("new_TWS_Pred_Input1.csv")
df1.dropna(inplace=True)
df1

,Month,Longitude,Latitude,TWS_tavg_mean,aet_sum,precipitation,ro_sum,soil_mean,tmmn_mean,tmmx_mean,Date,cluster_id
0,2003_02,73.661853,31.575782,657.42566,603.0,92.135190,3.0,21.0,84.0,222.0,2003-02-01,2
1,2003_02,73.706769,31.575782,657.42566,602.0,93.187584,3.0,24.0,85.0,222.0,2003-02-01,2
2,2003_02,73.751685,31.575782,817.91895,604.0,90.842030,3.0,25.0,85.0,222.0,2003-02-01,2
3,2003_02,73.796601,31.575782,817.91895,605.0,82.646610,3.0,55.0,85.0,222.0,2003-02-01,2
4,2003_02,73.841516,31.575782,817.91895,605.0,79.341210,3.0,57.0,85.0,222.0,2003-02-01,2
...,...,...,...,...,...,...,...,...,...,...,...,...
9607271,2023_12,74.200842,21.694314,1138.94210,135.0,0.000000,0.0,369.0,129.0,303.0,2023-12-01,5
9607272,2023_12,74.245758,21.694314,1143.05370,129.0,5.335234,0.0,353.0,135.0,315.0,2023-12-01,5
9607273,2023_12,74.290674,21.694314,1143.05370,124.0,6.006587,0.0,341.0,136.0,316.0,2023-12-01,5
9607274,2023_12,74.335590,21.694314,1143.05370,119.0,7.141133,0.0,327.0,139.0,321.0,2023-12-01,5


In [13]:
import pandas as pd

# First, standardize column names in df to match df1
df = df.rename(columns={
    'longitude': 'Longitude',
    'latitude': 'Latitude',
    'date': 'Date'
})

# Merge on Longitude, Latitude, and Date
merged_df = pd.merge(
    df1[['Longitude', 'Latitude', 'Date', 'TWS_tavg_mean']],
    df[['Longitude', 'Latitude', 'Date', 'value']],
    on=['Longitude', 'Latitude', 'Date'],
    how='inner'
)

# Optional: Rename columns for clarity
merged_df = merged_df.rename(columns={
    'TWS_tavg_mean': 'TWS',
    'value': 'predTWS'
})

print(merged_df.head())


Empty DataFrame
Columns: [Longitude, Latitude, Date, TWS, predTWS]
Index: []


In [17]:
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree

# Create coordinate arrays
coords_df1 = df1[['Longitude', 'Latitude']].to_numpy()
coords_df = df[['longitude', 'latitude']].to_numpy()

# Build KDTree from df1
tree = cKDTree(coords_df1)

# Query nearest neighbors in df1 for each point in df
distances, indices = tree.query(coords_df, k=1)

# Create a new DataFrame that includes the nearest matches
df_nn = df.copy()
df_nn['nn_index'] = indices
df_nn['Distance'] = distances

# Add df1 index to keep track of matched rows
df1 = df1.reset_index(drop=True)
df_nn = df_nn.reset_index(drop=True)

# Join the nearest Longitude/Latitude from df1 into df_nn
df_nn['Matched_Longitude'] = df1.loc[df_nn['nn_index'], 'Longitude'].values
df_nn['Matched_Latitude'] = df1.loc[df_nn['nn_index'], 'Latitude'].values
df_nn['Matched_Date'] = df_nn['date']  # prepare for merge on date

# Now merge on matched coordinates and Date
df1 = df1.rename(columns={'Date': 'Matched_Date'})
merged_df = pd.merge(
    df_nn,
    df1[['Longitude', 'Latitude', 'Matched_Date', 'TWS_tavg_mean']],
    left_on=['Matched_Longitude', 'Matched_Latitude', 'Matched_Date'],
    right_on=['Longitude', 'Latitude', 'Matched_Date'],
    how='inner'
)

# Final cleanup
merged_df = merged_df.rename(columns={
    'value': 'predTWS',
    'TWS_tavg_mean': 'TWS'
})[[
    'Matched_Longitude', 'Matched_Latitude', 'Matched_Date', 'TWS', 'predTWS', 'Distance'
]].rename(columns={
    'Matched_Longitude': 'Longitude',
    'Matched_Latitude': 'Latitude',
    'Matched_Date': 'Date'
})

print(merged_df.head())


   Longitude   Latitude        Date        TWS     predTWS  Distance
0  73.796601  31.530866  2003-02-01  817.91895  837.575928   0.03176
1  73.886432  31.530866  2003-02-01  817.91895  833.531860   0.03176
2  73.931348  31.530866  2003-02-01  817.91895  814.328064   0.03176
3  73.976264  31.530866  2003-02-01  817.91895  807.593262   0.03176
4  73.796601  31.485951  2003-02-01  799.58210  813.395996   0.03176


In [18]:
import numpy as np
from sklearn.metrics import r2_score

# Drop rows with NaNs (if any)
clean_df = merged_df.dropna(subset=['TWS', 'predTWS'])

# Correlation (Pearson)
correlation = clean_df['TWS'].corr(clean_df['predTWS'])

# R² Score
r2 = r2_score(clean_df['TWS'], clean_df['predTWS'])

print(f"Correlation (Pearson): {correlation:.4f}")
print(f"R² Score: {r2:.4f}")


Correlation (Pearson): 0.9481
R² Score: 0.8982
